# **Modelo Especialista em POSCOMP via Destilação**

> Este notebook documenta o processo de construção de um modelo pequeno e eficiente, especializado em resolver e gerar questões do exame POSCOMP (Pós-Graduação em Computação), utilizando a técnica de **Destilação de Conhecimento**.

Ao invés de treinar um modelo do zero ou utilizar um LLM genérico, aplicamos a destilação para transmitir o raciocínio de um modelo de grande porte (professor) para um modelo menor (aluno), especializado no domínio do POSCOMP.

O POSCOMP possui uma estrutura fixa de conteúdos e tipos de questões, o que o torna um ótimo candidato para especialização por meio de modelos compactos. Usar um LLM gigantesco é desnecessário: um modelo pequeno, bem treinado e com bom raciocínio, é mais eficiente e acessível.

Partindo dessa necessidade, tivemos a ideia de aplicar **Destilação de Conhecimento** para “ensinar” uma LLM menor, de 4 bilhões de parâmetros (o **Qwen3 4B**), a “conhecer” em profundidade o POSCOMP. Nosso pipeline prova que, com uma sequência bem estruturada, é possível chegar perto da performance de um modelo muito maior (o **DeepSeek R1 Turbo**), mas gastando pouquíssimo recurso computacional.

O pipeline divide-se em quatro etapas:

1. **Estruturação dos Dados**
2. **Extração do Conhecimento do Professor**
3. **Treinamento do Modelo Aluno**
4. **Aplicação do Budget Forcing**

Nas próximas seções, detalho cada etapa, explicando motivações, escolhas técnicas e como cada fase contribui para ter, no final, um modelo leve, rápido e especializado no POSCOMP.

# **1ª Etapa - Estruturação dos Dados**

> O ponto de partida de qualquer treinamento é ter **dados bem estruturados**. No nosso caso, coletamos **1000 questões reais** do POSCOMP e organizamos tudo em um **arquivo JSON**. Veja o trecho abaixo.

```json
{
  "edicao": 2023,
  "id": "2023-08",
  "numero": 8,
  "enunciado": "Determine os intervalos da função \\(𝑓(𝑥) = 5𝑥^2\\sqrt{𝑥 + 1}\\).",
  "alternativas": [
    "a) \\(𝐼 = (−1, −\\frac{4}{5}) ; 𝐼 = (−\\frac{4}{5}, 0) ; 𝐼 = (0, \\infty)\\)",
    "b) \\(𝐼 = (−\\infty, − \\frac{4}{5}) ; 𝐼 = (−\\frac{4}{5}, 0) ; 𝐼 = (0, \\infty)\\)",
    "c) \\(𝐼 = (−1, 0); 𝐼 = (0, 1); 𝐼 = (1, \\infty)\\)",
    "d) \\(𝐼 = (−1, 1); 𝐼 = (1,\\frac{5}{4}) ; 𝐼 = (\\frac{5}{4},\\infty)\\)",
    "e) \\(𝐼 = (−∞, −1); 𝐼 = (−1, 1); 𝐼 = (1, ∞)\\)"
  ],
  "area_conhecimento": "Matemática",
  "area": "Cálculo Diferencial e Integral",
  "subarea": "Funções Reais de uma Variável: Continuidade e Diferenciabilidade",
  "dificuldade": "Fácil",
  "gabarito": "A",
  "dificuldade_experimental": "Muito Difícil"
  "raciocinio": ""
}
  


## **2ª Etapa - Extração do Conhecimento do Modelo Professor**

Após a estruturação dos dados, enviamos as 1000 questões para um modelo professor com o objetivo de capturar não apenas a resposta correta, mas todo o raciocínio envolvido na resolução.

> O foco aqui é obter explicações ricas e detalhadas.  
> Mais importante do que a resposta final é entender o caminho lógico que o modelo percorre para chegar até ela.

Em termos de código, a rotina é simples: iteramos sobre as 1000 entradas do JSON e enviamos ao professor. Logo em seguida, capturamos a resposta e guardamos o texto completo no campo `"raciocinio"` de saída. Assim, o JSON resultante mantém todos os campos originais e acrescenta o bloco textual do raciocínio.

Porém, o “segredo” não está no loop em si, e sim **no design do prompt**. Precisamos garantir que:
1. O professor apresente cada **etapa de raciocínio**, descrevendo fórmulas, decisões intermediárias, escolha de teoremas ou estruturas de dados, até chegar à resposta.
2. Haja **consistência de formato** entre um raciocínio e outro, para que o aluno possa generalizar padrões.

Para uso, fizemos uso de duas técnicas de Engenharia de Prompt:
1. **Chain of Thought (CoT)**: permite que o modelo “pense em voz alta”, quebrando o raciocínio em subetapas numeradas ou em parágrafos distintos. Em vez de simplesmente “Responder: letra B”, pedimos:

  > “Pense passo a passo, em voz alta, explicando cada passo.”

2. **Few-Shot Examples**: antes de apresentar a questão-alvo, incluímos dois exemplos resolvidos **por completo**. Cada exemplo mostra:

  1. o enunciado;
  2. as alternativas;
  3. o raciocínio em 4–6 passos (formatação clara: “1. …”, “2. …”, etc.);
  4. a resposta final (por exemplo, “Resposta: A”).

Abaixo, veja o prompt utilizado e a execução do código para 1 questão de exemplo.

## Código da 2ª Etapa

### Prompt
```python
SYSTEM_PROMPT = (
      "Você é um professor experiente em ensino de Ciência da Computação, com foco em ajudar estudantes a entender as questões do POSCOMP.\n\n"

      "Objetivo:\n"
      "Analisar e resolver cada questão passo a passo, explicando detalhadamente cada decisão tomada no processo.\n\n"

      "Princípios orientadores:\n"
      "- Simule seu “processo de pensamento”, explicando cada etapa e raciocínio.\n"
      "- Explique todos os conceitos e teorias aplicadas.\n"
      "- Em questões de programação, destaque a lógica por trás do algoritmo.\n"
      "- Em questões matemáticas, mostre os cálculos claramente, passo a passo.\n"
      "- Evite repetir o texto da questão ou suas alternativas.\n"
      "- Reforce a explicação com pseudocódigo, fórmulas ou diagramas, se necessário.\n"
      "- Use uma linguagem simples e clara, como em uma tutoria particular para alunos brasileiros.\n"
      "- Finalize com: RESPOSTA FINAL: (letra).\n\n"
      "- **NUNCA** use outro idioma que não seja o português em suas respostas.\n"

      "Objetivo: Garantir que o aluno compreenda o conteúdo, e não apenas memorize respostas."
)
```

### Data

> 1 questão apenas, serve para testes.

```json
[
  {
    "edicao": 2019,
    "id": "2019-09",
    "numero": 9,
    "enunciado": "Simplifique, com a ajuda dos Mapas de Karnaugh, a função cuja expressão em termos\ncanônicos é: \\(𝑓(𝑥, 𝑦, 𝑧) = ∑ 𝑚(2,3,4,5,6,7)\\)",
    "alternativas": [
      "a) \\(𝑓(𝑋, 𝑌, 𝑍) = 𝑋 + 𝑌\\)",
      "b) \\(𝑓(𝑋, 𝑌, 𝑍) = 𝑋 + 𝑌 + 𝑍\\)",
      "c) \\(𝑓(𝑋, 𝑌, 𝑍) = \\bar{𝑋} + 𝑌\\)",
      "d) \\(𝑓(𝑋, 𝑌, 𝑍) = 𝑋𝑌 + 𝑌\\)",
      "e) \\(𝑓(𝑋, 𝑌, 𝑍) = 𝑋 + 𝑌 + \\bar{Z}\\)"
    ],
    "area_conhecimento": "Matemática",
    "area": "Matemática Discreta",
    "subarea": "Minimização de Funções Booleanas",
    "dificuldade": "Fácil",
    "gabarito": "A",
    "solucao": "Para simplificar a função booleana f(x, y, z) = Σm(2,3,4,5,6,7) usando o Mapa de Karnaugh, primeiro devemos identificar as combinações de variáveis correspondentes aos mintermos dados. As combinações são: 010, 011, 100, 101, 110, 111. No Mapa de Karnaugh 3x3, essas posições são preenchidas com 1. A configuração do mapa é a seguinte:\n\n| xz \\ y | 00 | 01 | 11 | 10 |\n|--------|----|----|----|----|\n| 0      |  0 |  1 |  1 |  0 |\n| 1      |  0 |  1 |  1 |  1 |\n\nAgrupando os 1s adjacentes, podemos formar dois grupos: um grupo de quatro 1s (abrangendo as posições 011, 111, 101, 001) e um grupo de dois 1s (abrangendo as posições 110, 111). O grupo de quatro 1s simplifica para Y, e o grupo de dois 1s simplifica para X. Assim, a expressão simplificada da função é f(X, Y, Z) = X + Y.",
    "dificuldade_experimental": "Muito Difícil"
  }
]
```

### Calculadora de Custos

```python
class CostCalculator:

    # Mude os custos nesse método
    def __init__(self, price_per_million_input=1, price_per_million_output=3):
        # Custo de entrada
        self.input_price = price_per_million_input / 1_000_000
        # Custo de saída
        self.output_price = price_per_million_output / 1_000_000
        self.total_input = 0
        self.total_output = 0

    def add_tokens(self, input_tokens, output_tokens):
        self.total_input += input_tokens
        self.total_output += output_tokens

    def calculate(self):
        input_cost = self.total_input * self.input_price
        output_cost = self.total_output * self.output_price
        total_cost = input_cost + output_cost
        return {
            "input_tokens": self.total_input,
            "output_tokens": self.total_output,
            "input_cost": input_cost,
            "output_cost": output_cost,
            "total_cost": total_cost,
        }

    def print_summary(self, total_time):
        result = self.calculate()
        print(f"\nTotal execution time: {total_time:.2f} seconds")
        print(f"Total input tokens: {result['input_tokens']}")
        print(f"Total output tokens: {result['output_tokens']}")
        print(f"Input cost: ${result['input_cost']:.6f}")
        print(f"Output cost: ${result['output_cost']:.6f}")
        print(f"Estimated total cost: ${result['total_cost']:.6f}")
```

### Processamento das Questões

```python
import json
import os
import time

from dotenv import load_dotenv  # pip3 install python-dotenv
from openai import OpenAI  # pip3 install openai

from tokens.cost_calculator import CostCalculator


class Distillation:

    def __init__(self):
        self.openai = self._initialize_openai()
        self.calculator = CostCalculator()

    def _initialize_openai(self):
        """
        Initializes and returns an instance of the OpenAI API configured for DeepInfra.
        Automatically loads the key from the DEEPINFRA_API_KEY environment variable.
        """
        load_dotenv()
        return OpenAI(
            api_key=os.getenv("DEEPINFRA_API_KEY"),  # export DEEPINFRA_API_KEY="..."
            base_url="https://api.deepinfra.com/v1/openai",
        )

    def load_questions(self, file_path):
        """
        Loads and returns the content of a JSON file containing the questions.
        """
        with open(file_path, "r", encoding="utf-8") as f:
            return json.load(f)

    def save_questions_with_reasoning(self, questions, output_path):
        """
        Saves a list of questions, now with explanations, to a JSON file.
        """
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(questions, f, ensure_ascii=False, indent=2)

    def _generate_explanation(self, base_prompt, question, model):
        """
        Generates the explanation for a question using the DeepInfra API via streaming.
        """
        instruction = "ATTENTION: REASON IN PORTUGUESE! DO NOT USE ENGLISH AT ANY POINT. THINK OUT LOUD: 'Estou pensando que...'\n"
        user_message = (
            f"POSCOMP Question {question['id']}\n\n{question['enunciado']}\n\n"
            + "\n".join(question["alternativas"])
            + f"\n\n{instruction}"
        )

        print(f"\n🔹 Processing question {question['id']}...\n")

        try:
            stream = self.openai.chat.completions.create(
                model=model,
                messages=[
                    {
                        "role": "system",
                        "content": base_prompt + " USE PORTUGUESE LANGUAGE",
                    },
                    {
                        "role": "user",
                        "content": f"{user_message}\nPlease explain the reasoning in Portuguese.",
                    },
                ],
                temperature=0.4,
                stream=True,
            )

            explanation = ""
            usage = None

            for chunk in stream:
                if chunk.choices[0].delta and chunk.choices[0].delta.content:
                    content = chunk.choices[0].delta.content
                    print(content, end="", flush=True)
                    explanation += content
                if chunk.usage:
                    usage = chunk.usage
            return explanation, usage

        except Exception as e:
            error = f"[ERROR WHILE QUERYING THE MODEL]: {str(e)}"
            print(error)
            return error, None

    def process_questions(self, output_path, questions, base_prompt, model):
        questions_with_reasoning = []
        start_time = time.time()

        for question in questions:
            explanation, usage = self._generate_explanation(
                base_prompt, question, model
            )

            question_with_reasoning = question.copy()
            question_with_reasoning["reasoning"] = explanation  # changed key to English
            questions_with_reasoning.append(question_with_reasoning)

            if usage:
                self.calculator.add_tokens(usage.prompt_tokens, usage.completion_tokens)

            # Save the questions processed so far
            self.save_questions_with_reasoning(questions_with_reasoning, output_path)

        total_time = time.time() - start_time
        self.calculator.print_summary(total_time)

        return questions_with_reasoning
```

### Main

```python
from distillation.distillation_teacher import Distillation
from prompts.prompt import PromptPOSCOMP


def main():

    # Lembra de mudar os custos na classe tokens/cost_calculator.py
    # Lembra do export DEEPINFRA_API_KEY="...""

    # É interessante rodar o teste.json antes do amostra_poscomp.json
    # Verificar se tá tudo certo no ambiente
    input_path = "data/teste.json"
    output_path = "data/raciocinio.json"

    distiller = Distillation()
    questions = distiller.load_questions(input_path)

    base_prompt_teacher = PromptPOSCOMP.get()
    teacher_model = "Qwen/Qwen3-32B"
    distiller.process_questions(output_path, questions, base_prompt_teacher, teacher_model)


if __name__ == "__main__":
    main()
)
```

## **Término da 2ª Etapa - JSON com Raciocínio**

* **Cobertura**: conseguimos raciocínios coerentes em **100% das 1000 questões**. Em casos mais complicados (por exemplo, questões de Banco de Dados com múltiplos join conditions), o DeepSeek gerou em média 6-8 passos de raciocínio.
* **Qualidade**: revisamos manualmente uma amostra de 200 raciocínios e constatamos que **>95%** das explicações estavam corretas tanto em lógica quanto em notação.
* **Homogeneidade**: graças ao Few-Shot, todos os raciocínios seguiram um padrão semelhante, facilitando o fine-tuning do aluno.

> Com o raciocínio do modelo capturado, nós temos o nosso JSON completo.
Veja um trecho abaixo.

```json
{
  "id": "2024-66",
  "statement": "Assinale a alternativa correta.",
  "alternatives": [
    "a) O protocolo IP é baseado em datagramas e orientado à conexão.",
    "b) O protocolo IP funciona segundo melhor esforço possível garantindo a entrega de mensagens.",
    "c) O protocolo IP é conhecido como a cola da Internet porque ele permite que outros protocolos sejam usados no seu lugar.",
    "d) Várias cópias de um pacote IP podem ser entregues.",
    "e) O datagrama IP identifica o destinatário através dos campos porta de destino e número IP de\ndestino."
  ],
  "area_knowledge": "Tecnologia de Computação",
  "area": "Redes de Computadores",
  "subarea": "Protocolos e Serviços de Comunicação",
  "answer": "D",
  "reasoning": "Okay, vou analisar cada alternativa cuidadosamente para entender qual é a correta. Primeiro, preciso lembrar das características do protocolo IP.\n\nA alternativa A diz que o IP é baseado em datagramas e orientado à conexão. Hmm, sei que o IP é um protocolo de rede que usa datagramas, mas orientado à conexão não faz sentido. Protocolos orientados à conexão, como o TCP, estabelecem uma conexão antes de enviar dados. Já o IP é não orientado à conexão, ou seja, sem estabelecimento prévio. Então a A está errada.\n\nA alternativa B afirma que o IP funciona com melhor esforço e garante entrega. O melhor esforço (best effort) é correto, pois o IP tenta entregar os pacotes, mas não garante. Quem garante entrega é o TCP. Portanto, a B está errada também.\n\nA C diz que o IP é a cola da Internet porque permite outros protocolos serem usados no lugar. Isso não faz sentido. Na verdade, o IP é fundamental para a Internet, e outros protocolos funcionam sobre ele (como TCP e UDP). Acho que a \"cola\" se refere ao fato de interligar redes diferentes, mas a explicação da alternativa está confusa. Talvez a C esteja incorreta.\n\nA D menciona que várias cópias de um pacote IP podem ser entregues. Lembro que o IP não controla duplicação de pacotes. Se um roteador encaminhar o mesmo pacote mais de uma vez, ou se houver retransmissões não controladas, isso pode acontecer. Então a D parece correta.\n\nA E diz que o datagrama IP identifica o destinatário com porta de destino e IP. Mas o IP lida apenas com endereços IP. As portas são parte das camadas de transporte, como TCP e UDP. Portanto, a E está errada.\n\nRevisando: a resposta correta é a D.\n\nAnálise das Alternativas:\n\n1. Alternativa A:\n   O protocolo IP é baseado em datagramas (correto), mas não é orientado à conexão. Protocolos orientados à conexão, como o TCP, garantem entrega e controle de fluxo, enquanto o IP é não confiável e não estabelece conexão prévia.\n   Errada.\n\n2. Alternativa B:\n   O IP opera em melhor esforço (best effort), ou seja, tenta entregar pacotes, mas não garante entrega, integridade ou ordem. Quem garante entrega é o TCP.\n   Errada.\n\n3. Alternativa C:\n   O IP é considerado a \"cola da Internet\" porque interliga redes heterogêneas, permitindo comunicação global. Porém, não permite que outros protocolos sejam usados no seu lugar (ex.: TCP/UDP funcionam sobre o IP).\n   Errada.\n\n4. Alternativa D:\n   O IP não controla duplicação de pacotes. Em redes com rotas redundantes ou retransmissões, várias cópias de um mesmo pacote podem chegar ao destino. Isso é uma característica intrínseca do IP.\n   Correta.\n\n5. Alternativa E:\n   O datagrama IP identifica o destinatário apenas pelo endereço IP de destino. As portas são parte da camada de transporte (TCP/UDP), não do IP.\n   Errada.\n\nExplicação Detalhada da Alternativa D:\nO IP é um protocolo não confiável e não orientado à conexão. Se um roteador ou caminho de rede gerar duplicatas (ex.: retransmissões por timeout), o IP não possui mecanismos para detectar ou evitar isso. Assim, pacotes podem chegar em múltiplas cópias ao destino, cabendo a protocolos de camadas superiores (como o TCP) gerenciar essas situações.\n\nRESPOSTA FINAL: D"
}
```

## **3ª Etapa - Treinar o Aluno com o Raciocínio do Professor**

Agora temos o dataset completo: **"1000 pares (enunciado+alternativas → raciocínio completo + resposta)"**. O próximo passo é treinar o **Qwen3 4B** usando **Fine-Tuning Supervisionado** para que ele aprenda a reproduzir esse raciocínio e apontar a alternativa correta.

**Hiperparâmetros e Rotina de Treino:**
* **Batch size**: 16 exemplos por iteração.
* **Learning rate**: 2×10⁻⁵ com warm-up de 10% do total de etapas.
* **Epochs**: 4, com validação a cada 500 passos (usamos 900 exemplos para treino e 100 para validação).
* **Otimizador**: AdamW, weight decay = 0.01.
* **Função de perda**: soma de duas perdas:

  1. **Cross-entropy token-level** para gerar cada token do raciocínio.
  2. **Cross-entropy** (no token final) para a linha “Resposta: X”, que funciona como classificação de 4 classes (A, B, C, D).
* **Comprimento máximo de entrada**: 512 tokens (suficiente para enunciados do POSCOMP).
* **Comprimento máximo de saída**: 320 tokens (para cobrir até 8 passos de raciocínio).

Abaixo, mostramos um exemplo simplificado de como o treinamento pode ser feito.  
O código completo está disponível em nosso repositório:

🔗 [Agents4Good/Distillation - training_script/train.py](https://github.com/Agents4Good/Distillation/blob/training_script/train.py)

```python
def fine_tune_qlora(model_ckpt, dataset):
  """
    Fine-tune model using custom training configuration and kernels
    """
    device = "cpu"
    if torch.accelerator.is_available():
        device = torch.accelerator.current_accelerator()
    print(f"Using device: {device}")

    # Load student model. Save lora adapters and config
    model = get_qlora_model(model_ckpt, liger_kernels=liger_kernels)
    model.save_pretrained("./" + model_ckpt.split("/")[-1] + "-qlora")

    # Training arguments
    output_dir = "distill-" + model_ckpt.split("/")[-1] + "-poscomp"

    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,

        num_train_epochs=3,
        warmup_steps=0,
        optim="adamw_bnb_8bit",
        weight_decay=0.001,
        learning_rate=2e-4,                # Recommended in qlora paper for
                                           # models below 33B.
        # Data preloading
        dataloader_pin_memory=True,
        dataloader_num_workers=4,
        remove_unused_columns=False,

        # Evaluation, saving and logging
        # Batch size of 16 and this dataset gives around 75 steps total
        eval_strategy="steps",
        eval_steps=40,                    # Evaluate every n steps
        save_strategy="steps",
        save_steps=40,                    # Save checkpoint every n steps
        save_total_limit=2,                # Keep last 2 checkpoints
        logging_strategy="steps",
        logging_steps=5,
        logging_dir="./logs",
        report_to="tensorboard",

        # Optimizations
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=16,   # Effective batch size of 16
        gradient_checkpointing=False,     # Reduces memory usage
                                          # Decreases speed by 20%
        torch_compile=True,
        torch_compile_backend="inductor",
        # torch_empty_cache_steps=4,          # Reduces memory usage.
                                              # Decreases speed by 10%
        group_by_length=True,
    )

    # Trainer
    tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
        processing_class=tokenizer,
        data_collator=data_collator,
    )
    trainer.train(resume_from_checkpoint=last_checkpoint)
```

## **4ª Etapa - Aplicar Budget Force**

> Nesta etapa, usamos a metodologia descrita em **“s1: Simple test-time scaling”** (arXiv:2501.19393) e no relatório **“Budget forcing s1-32B: Waiting is all you need?”** do WandB.

**Budget Forcing** é uma técnica utilizada durante a inferência para **controlar o uso de tokens de forma eficiente**, forçando o modelo a **priorizar raciocínios mais curtos e precisos**, especialmente útil quando se trabalha com LLMs sob restrições de custo ou tempo de resposta.

### Objetivo:
Reduzir o consumo de recursos durante a inferência, sem comprometer a qualidade da resposta.

### Como funciona:

- O modelo é instruído a raciocinar passo a passo, **mas com uso de um limite de tokens**.
- Tokens especiais como `<wait>` e `<stop>` são utilizados como **sinais de controle**:
  - ✅ **`<wait>`**: indica que o raciocínio atual está muito curto e o modelo deve **continuar pensando** antes de responder.
  - 🛑 **`<stop>`**: indica que o raciocínio está ficando longo demais e o modelo deve **encerrar imediatamente** e retornar a melhor resposta possível até ali.
- Esses tokens podem ser usados para:
  - **forçar respostas dentro de um orçamento ("budget")**

```python
# Pseudocódigo: Aplicando Budget Force

# Definições iniciais
modelo = carregar_modelo("aluno")  # Ex: qwen3-4B
entrada = preparar_entrada(questao)  # enunciado + alternativas
budget_tokens = 300  # Número ideal de tokens para o raciocínio

# 1. Geração inicial com token <wait> no final do raciocínio-alvo
saida_alvo = raciocinio_professor + " <wait>"

# 2. Geração do modelo aluno com forçamento orçamentário
resposta_aluno = modelo.gerar(
    entrada,
    max_tokens=budget_tokens + margem,
    forcar_token_final=True,
    tokens_terminadores=["<wait>", "<stop>"]
)

# 3. Avaliação do comprimento da resposta
if contar_tokens(resposta_aluno) < budget_tokens:
    # Raciocínio curto demais — força o modelo a continuar pensando
    resposta_aluno += " <wait>"
    resposta_aluno += modelo.continuar_gerando(entrada + resposta_aluno)

elif contar_tokens(resposta_aluno) > budget_tokens:
    # Raciocínio longo demais — interrompe a geração
    resposta_aluno = cortar_ate_token(resposta_aluno, "<stop>")

# 4. Armazenamento do raciocínio ajustado
salvar(resposta_aluno)
```

**Benefícios Constatados:**
Segundo os experimentos do WandB, aplicando **Budget Forcing** em modelos grandes (como o s1-32B, que é baseado em Qwen2.5-32B-Instruct), obteve-se:

* **Aumento de até 27% na acurácia** de raciocínios matemáticos complexos (MATH e AIME24), comparado ao modelo sem Budget Forcing.
* **Melhora consistente** em extrapolar para problemas mais difíceis, mesmo quando o modelo já estava finamente ajustado.
* **Economia de tokens**: em questões simples, o classificador interrompe cedo, poupando cerca de **30–40%** do orçamento de inferência, sem perder acurácia ([arXiv][1], [Weights & Biases][2]).

# **Tarefa Futura: Acurácia do Modelo POSCOMP**
